## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/evlko/CS-493/main/Data/sms_spam.csv')

df['type'] = df['type'].map({'ham': 0, 'spam': 1})

y = df['type']
X = df.drop(columns=['type'])

### Предобработка

In [ ]:
import nltk
import re
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

In [4]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def preprocess(text):
    """Функция предобработки текста.
    Убираем всю пунктуацию;
    Приводим текст к нижнему регистру;
    Циклом проходимся по токенам и проверяем, есть ли их лемма в стоп-словах, если нет, то сохраняем лемму токена.
    """
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = [lemmatizer.lemmatize(word) for word in word_tokenize(text) if lemmatizer.lemmatize(word) not in stopwords] 
    text = ' '.join(text)

    return text

X['text'] = X['text'].apply(preprocess)

### Построение модели

In [5]:
from sklearn.naive_bayes import MultinomialNB 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import ngrams

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

# dataframe для сравнения векторайзеров
df_scores = pd.DataFrame(columns=['vectorizer', 'precision', 'recall', 'f1-score','accuracy'])

#### мешок n-грамм

In [7]:
vectorizer_name = 'bag of word n-grams'

In [ ]:
# создаем экземпляр pipeline - последовательность шагов предобработки и модель
pipeline = Pipeline([
           ('ngram', CountVectorizer()),
           ('clf', MultinomialNB()),
])

# константы данного классификатора, будут использоваться для настройки параметров GridSearchCV
MIN_NGRAM = 1
MAX_NGRAM = 10

n_grams = [(i, j) for i in range(MIN_NGRAM, MAX_NGRAM + 1) for j in range(MIN_NGRAM + 1, MAX_NGRAM + 1)]

# создаем словарь параметров
parameters = {
    'ngram__ngram_range': n_grams,
}

# создаем экземпляр сетки поиска наилучших гиперпараметров
grid_search = GridSearchCV(pipeline, parameters, cv=2, verbose=1, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train['text'], y_train)

In [9]:
print(f"Лучший диапазон n-gram слов: {grid_search.best_params_['ngram__ngram_range']}.")

Лучший диапазон n-gram слов: (1, 2).


In [10]:
predictions = grid_search.predict(X_test['text'])

# в метриках берем средневзвешенное, так как есть имбаланс классов
precision, recall, f1score, support = precision_recall_fscore_support(y_test, predictions, average='weighted')
acc = accuracy_score(y_test, predictions)
scores = [vectorizer_name, precision, recall, f1score, acc]

df_scores.loc[len(df_scores)] = scores

#### tf-idf

In [11]:
vectorizer_name = 'tf-idf'

In [ ]:
pipeline = Pipeline([
           ('tfidf', TfidfVectorizer()),
           ('clf', MultinomialNB()),
])

MIN_NGRAM = 1
MAX_NGRAM = 6

n_grams = [(i, j) for i in range(MIN_NGRAM, MAX_NGRAM + 1) for j in range(MIN_NGRAM, MAX_NGRAM + 1)]

MIN_MIN_DF = 0
MAX_MIN_DF = 0.01
STEP_MIN_DF = 0.005

MIN_MAX_DF = 0.5
MAX_MAX_DF = 1.0
STEP_MAX_DF = 0.25

MIN_MAX_FEATURES = 5000
MAX_MAX_FEATURES = 10000
STEP_MAX_FEATURES = 1000

parameters = {
    'tfidf__ngram_range': n_grams,
    'tfidf__min_df': np.arange(MIN_MIN_DF, MAX_MIN_DF + STEP_MIN_DF, STEP_MIN_DF),
    'tfidf__max_df': np.arange(MIN_MAX_DF, MAX_MAX_DF + STEP_MAX_DF, STEP_MAX_DF),
    'tfidf__smooth_idf': [True],
    'tfidf__max_features': np.arange(MIN_MAX_FEATURES, MAX_MAX_FEATURES + STEP_MAX_FEATURES, STEP_MAX_FEATURES),
}

grid_search = GridSearchCV(pipeline, parameters, cv=2, verbose=1, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train['text'], y_train)

In [13]:
print(f"Лучшие параметы:\nдиапазон n-gram слов: {grid_search.best_params_['tfidf__ngram_range']},\nmin df: {grid_search.best_params_['tfidf__min_df']},\nmax df: {grid_search.best_params_['tfidf__max_df']}\nmax features: {grid_search.best_params_['tfidf__max_features']}")

Лучшие параметы:
диапазон n-gram слов: (1, 2),
min df: 0.005,
max df: 0.5
max features: 5000


In [14]:
predictions = grid_search.predict(X_test['text'])

precision, recall, f1score, support = precision_recall_fscore_support(y_test, predictions, average='weighted')
acc = accuracy_score(y_test, predictions)
scores = [vectorizer_name, precision, recall, f1score, acc]

df_scores.loc[len(df_scores)] = scores

#### символьные n-граммы

In [15]:
vectorizer_name = 'bag of char n-grams'

In [ ]:
pipeline = Pipeline([
           ('ngram', CountVectorizer()),
           ('clf', MultinomialNB()),
])

MIN_NGRAM = 1
MAX_NGRAM = 6

n_grams = [(i, j) for i in range(MIN_NGRAM, MAX_NGRAM + 1) for j in range(MIN_NGRAM, MAX_NGRAM + 1)]

parameters = {
    'ngram__ngram_range': n_grams,
    'ngram__analyzer': ['char'],
}

grid_search = GridSearchCV(pipeline, parameters, cv=2, verbose=1, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train['text'], y_train)

In [17]:
print(f"Лучший диапазон n-gram символов: {grid_search.best_params_['ngram__ngram_range']}.")

Лучший диапазон n-gram символов: (3, 3).


In [18]:
predictions = grid_search.predict(X_test['text'])

precision, recall, f1score, support = precision_recall_fscore_support(y_test, predictions, average='weighted')
acc = accuracy_score(y_test, predictions)
scores = [vectorizer_name, precision, recall, f1score, acc]

df_scores.loc[len(df_scores)] = scores

### scores

In [19]:
df_scores

,vectorizer,precision,recall,f1-score,accuracy
0,bag of word n-grams,0.983622,0.983813,0.983617,0.983813
1,tf-idf,0.966832,0.967626,0.966574,0.967626
2,bag of char n-grams,0.981456,0.981415,0.981435,0.981415


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [20]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [21]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

потому что пересекается

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [22]:
# \b - word-boundary, \w - [A-Za-z0-9_]
result = re.findall(r'\b(\w\w?)', 'It is more than a university')
print(result)

['It', 'is', 'mo', 'th', 'a', 'un']


### split
разделяет строку по заданному шаблону


In [23]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [24]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [25]:
result = re.split('\.[^\b]', 'One. Two. Three. Four. Five.', maxsplit=2)
print(result)

['One', 'Two', 'Three. Four. Five.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [26]:
result = re.sub('a', 'b', 'abcabc')
print(result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [27]:
# \d - любая цифра
result = re.sub('\d+', 'DIG', '+7 (800) 555-35-35')
print(result)

+DIG (DIG) DIG-DIG-DIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [28]:
result = re.sub(r'(\w*:\/\/)?(www.)?[\w]+\.\w+(\/[\w]+\/?)*\b', '', 'It is more than itmo.ru or http://iTmo.ru or https://itmo.ru or even itmo.ru/home')
print(result)

It is more than  or  or  or even 


### compile
компилирует регулярное выражение в отдельный объект

In [29]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [30]:
rx = re.compile('[А-Яа-яё\-|A-Za-z]{4,}')
result = rx.findall('Слова? Да, больше, ещё больше слов! Что-то ещё. And English.')
print(result)

['Слова', 'больше', 'больше', 'слов', 'Что-то', 'English']


**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [31]:
# ?<=@ - если следует после @ с определенными условями 
rx = re.compile(r'(?<=@)(\w[\w-]*\w(?:\.\w[\w-]*\w)*)\b')
result = rx.findall('abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')
print(result)

['gmail.com', 'test.in', 'analyticsvidhya.com', 'rest.biz']
